In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
# Creación de objeto pandas datafr-ame
patients_df=pd.read_csv('https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv?raw=true')
patients_df.to_csv("./data.csv")

In [3]:
# Creación variables binarias
patients_df.replace({'sex':{'male':0,'female':1}}, inplace=True)
patients_df.replace({'smoker':{'yes':1,'no':0}}, inplace=True)

In [4]:
# La función get dummies convierte un DataFrame de columnas categoricas a uno con variables dummy variables
region_dummies_df=pd.get_dummies(patients_df[['region']])
region_dummies_df = region_dummies_df.replace().astype(int)
region_dummies_df

,region_northeast,region_northwest,region_southeast,region_southwest
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
1333,0,1,0,0
1334,1,0,0,0
1335,0,0,1,0
1336,0,0,0,1


In [5]:
# Hacemos join entre los 2 dataframes para reconstruir el dataset
patients_df = patients_df.join(region_dummies_df)

In [6]:
patients_df = patients_df.drop(['region'], axis=1)
patients_df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,1,16884.92400,0,0,0,1
1,18,0,33.770,1,0,1725.55230,0,0,1,0
2,28,0,33.000,3,0,4449.46200,0,0,1,0
3,33,0,22.705,0,0,21984.47061,0,1,0,0
4,32,0,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,10600.54830,0,1,0,0
1334,18,1,31.920,0,0,2205.98080,1,0,0,0
1335,18,1,36.850,0,0,1629.83350,0,0,1,0
1336,21,1,25.800,0,0,2007.94500,0,0,0,1


In [7]:
# Uso 70% para entrenamiento (random split)
train_df= patients_df.sample(frac=0.7,random_state=200)
rest_df = patients_df.drop(train_df.index)
# Uso 15% para validacion y 15% para test
val_df=rest_df.sample(frac=0.5,random_state=200)
test_df=rest_df.drop(val_df.index)

In [77]:

def polynomial_features(X, degree):
    from itertools import combinations_with_replacement
    n_samples, n_features = X.shape
    combs = [combinations_with_replacement(range(n_features), i) for i in range(0, degree + 1)]
    combs = [item for sublist in combs for item in sublist]
    n_output_features = len(combs)
    X_new = np.empty((n_samples, n_output_features))
    for i, index_combs in enumerate(combs):
        X_new[:, i] = np.prod(X[:, index_combs], axis=1)
    return X_new

class linear_regressor():
    '''
        Clase que implementa el algoritmo de regression logistica con opcion de anadir terminos polinomicos 
    '''
    def __init__(self, degree=1):
        
        self.theta = None
        self.degree = degree
        

    def fit_model(self, X, Y, lr=0.00001, epochs=100, patience=10, l2=None):
        X = polynomial_features(X, self.degree)
        self.l2 = l2
        n, m = X.shape
        self.theta = np.random.rand(m+1,1)
        X_c = np.hstack((np.ones((n,1)),X))
        loss_v = []
        best_loss = np.inf
        epochs_stall = 0
        
        for epoch in range(epochs):
            Y_est = X_c.dot(self.theta)
            
            # Calcular la funcion de perdida con regularizacion l2 o sin 
            loss = np.sum(np.power(Y_est-Y,2))/(2.*n) + (self.l2 * np.sum(np.square(self.theta)) / 2*m if self.l2 else 0)
            loss_v.append(loss)
            
            # Calcular gradientes con regularizacion l2 o sin             
            regularization_term = (self.l2/m) * self.theta if self.l2 else 0
            gradientes = (-1/n) * (X_c.T.dot((Y - Y_est))) + regularization_term
            
            # Update weights
            self.theta = self.theta - lr*gradientes

            # Esto se agrega para parar el entrenamiento en caso de que la perdida no disminuya por mas del valor del parametro patience
            if loss < best_loss:
                best_loss = loss
                best_theta = np.copy(self.theta)
                epochs_stall = 0
            else:
                epochs_stall += 1
                
            if epochs_stall >= patience:
                print('La funcion de perdida no ha disminuido, parando despues de {} epocas. el error es: {:.4e}'.format(epoch, loss))
                break
                
            print('Epoch: {} Loss: {:.4e}'.format(epoch, loss))
        
        print('El error final fue: {:.4e}'.format(loss))
        self.theta = best_theta
        self.loss_vector = loss_v

    def predict(self, X):
        X = polynomial_features(X, self.degree)
        X_c = np.hstack((np.ones((X.shape[0],1)),X))
        Y_hat = X_c.dot(self.theta)
        return Y_hat

Prueba de

In [83]:
LR = linear_regressor(3)
X = train_df.drop(['charges'], axis=1).values
Y = train_df[['charges']].values
x_test = test_df.drop(['charges'], axis=1).values
y_test = test_df['charges'].values
LR.fit_model(X, Y)

Epoch: 0 Loss: 8.5482e+09
Epoch: 1 Loss: 3.0887e+20
Epoch: 2 Loss: 1.1345e+31
Epoch: 3 Loss: 4.1673e+41
Epoch: 4 Loss: 1.5307e+52
Epoch: 5 Loss: 5.6226e+62
Epoch: 6 Loss: 2.0653e+73
Epoch: 7 Loss: 7.5862e+83
Epoch: 8 Loss: 2.7866e+94
Epoch: 9 Loss: 1.0236e+105
La funcion de perdida no ha disminuido, parando despues de 10 epocas. el error es: 3.7597e+115
El error final fue: 3.7597e+115


In [67]:
y_pred = LR.predict(x_test)

# mse
mse = np.sum(np.power(y_pred-y_test,2))/(2.*X.shape[0])
print("El error en test es de {:.4}".format(mse))

El error en test es de 2.998e+09


## Analisis de variables

### Multicolinealidad
Como se puede observar no existen variables altamente correlacionadas por lo tanto se decide utilizar todas las variables

In [73]:
correlation_matrix = np.corrcoef(X, rowvar=False)

# Crear un DataFrame con la matriz de correlación y las etiquetas de las columnas
correlation_df = pd.DataFrame(correlation_matrix, columns=patients_df.drop(columns=['charges']).columns, 
                              index=patients_df.drop(columns=['charges']).columns)

correlation_df

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
age,1.000000,0.027422,0.084431,0.060771,-0.051829,-0.010371,0.014073,-0.027887,0.024913
sex,0.027422,1.000000,-0.045460,-0.020528,-0.067844,0.019966,0.030021,-0.019838,-0.028997
bmi,0.084431,-0.045460,1.000000,0.029462,-0.001460,-0.134960,-0.162882,0.283670,0.003014
children,0.060771,-0.020528,0.029462,1.000000,-0.000314,-0.053117,0.034247,0.000312,0.018171
smoker,-0.051829,-0.067844,-0.001460,-0.000314,1.000000,0.045509,-0.049364,0.067197,-0.064973
region_northeast,-0.010371,0.019966,-0.134960,-0.053117,0.045509,1.000000,-0.316008,-0.340888,-0.323377
region_northwest,0.014073,0.030021,-0.162882,0.034247,-0.049364,-0.316008,1.000000,-0.342887,-0.325274
region_southeast,-0.027887,-0.019838,0.283670,0.000312,0.067197,-0.340888,-0.342887,1.000000,-0.350883
region_southwest,0.024913,-0.028997,0.003014,0.018171,-0.064973,-0.323377,-0.325274,-0.350883,1.000000


### Transformaciones

In [87]:
def log_transform(data):
    return np.log1p(data)

Y_log = log_transform(Y)

In [89]:
# Estandarización de características
def standardize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return (data - mean) / std, mean, std


X_standardized, mean_X, std_X = standardize(X)
Y_standardized, mean_Y, std_Y = standardize(Y)

model = linear_regressor()
model.fit_model(X_standardized, Y_standardized, lr=0.01, epochs=100)


Epoch: 0 Loss: 1.7232e+00
Epoch: 1 Loss: 1.6810e+00
Epoch: 2 Loss: 1.6402e+00
Epoch: 3 Loss: 1.6005e+00
Epoch: 4 Loss: 1.5619e+00
Epoch: 5 Loss: 1.5245e+00
Epoch: 6 Loss: 1.4881e+00
Epoch: 7 Loss: 1.4528e+00
Epoch: 8 Loss: 1.4185e+00
Epoch: 9 Loss: 1.3852e+00
Epoch: 10 Loss: 1.3528e+00
Epoch: 11 Loss: 1.3214e+00
Epoch: 12 Loss: 1.2908e+00
Epoch: 13 Loss: 1.2611e+00
Epoch: 14 Loss: 1.2322e+00
Epoch: 15 Loss: 1.2042e+00
Epoch: 16 Loss: 1.1769e+00
Epoch: 17 Loss: 1.1504e+00
Epoch: 18 Loss: 1.1246e+00
Epoch: 19 Loss: 1.0996e+00
Epoch: 20 Loss: 1.0752e+00
Epoch: 21 Loss: 1.0515e+00
Epoch: 22 Loss: 1.0284e+00
Epoch: 23 Loss: 1.0060e+00
Epoch: 24 Loss: 9.8421e-01
Epoch: 25 Loss: 9.6300e-01
Epoch: 26 Loss: 9.4236e-01
Epoch: 27 Loss: 9.2228e-01
Epoch: 28 Loss: 9.0275e-01
Epoch: 29 Loss: 8.8373e-01
Epoch: 30 Loss: 8.6523e-01
Epoch: 31 Loss: 8.4722e-01
Epoch: 32 Loss: 8.2969e-01
Epoch: 33 Loss: 8.1263e-01
Epoch: 34 Loss: 7.9602e-01
Epoch: 35 Loss: 7.7985e-01
Epoch: 36 Loss: 7.6410e-01
Epoch: 37 L

In [91]:
# Función para evaluar diferentes hiperparámetros en el conjunto de validación
def evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations):
    best_val_loss = float('inf')
    best_hyperparameters = None
    
    X_train = train_df.drop(columns=['charges']).values
    Y_train = train_df[['charges']].values
    X_val = val_df.drop(columns=['charges']).values
    Y_val = val_df[['charges']].values

    X_train, _, _ = standardize(X_train)
    X_val, _, _ = standardize(X_val)

    for lr in learning_rates:
        for degree in degrees:
            for reg in regularizations:
                model = linear_regressor(degree=degree)
                model.fit_model(X_train, Y_train, lr=lr, epochs=100, l2=reg)
                predictions = model.predict(X_val)
                val_loss = np.mean((predictions - Y_val)**2)
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_hyperparameters = (lr, degree, reg)
    
    return best_hyperparameters, best_val_loss

# Especificar rangos para hiperparámetros
learning_rates = [0.01, 0.001]
degrees = [1, 2, 3]
regularizations = [None, 0.1, 0.01]

best_hyperparameters, best_val_loss = evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations)

print("Mejores hiperparámetros:", best_hyperparameters)
print("Pérdida de validación con los mejores hiperparámetros:", best_val_loss)

Epoch: 0 Loss: 1.6038e+08
Epoch: 1 Loss: 1.5580e+08
Epoch: 2 Loss: 1.5138e+08
Epoch: 3 Loss: 1.4712e+08
Epoch: 4 Loss: 1.4300e+08
Epoch: 5 Loss: 1.3903e+08
Epoch: 6 Loss: 1.3519e+08
Epoch: 7 Loss: 1.3149e+08
Epoch: 8 Loss: 1.2792e+08
Epoch: 9 Loss: 1.2447e+08
Epoch: 10 Loss: 1.2114e+08
Epoch: 11 Loss: 1.1792e+08
Epoch: 12 Loss: 1.1482e+08
Epoch: 13 Loss: 1.1182e+08
Epoch: 14 Loss: 1.0892e+08
Epoch: 15 Loss: 1.0612e+08
Epoch: 16 Loss: 1.0342e+08
Epoch: 17 Loss: 1.0081e+08
Epoch: 18 Loss: 9.8283e+07
Epoch: 19 Loss: 9.5845e+07
Epoch: 20 Loss: 9.3489e+07
Epoch: 21 Loss: 9.1212e+07
Epoch: 22 Loss: 8.9011e+07
Epoch: 23 Loss: 8.6884e+07
Epoch: 24 Loss: 8.4827e+07
Epoch: 25 Loss: 8.2839e+07
Epoch: 26 Loss: 8.0917e+07
Epoch: 27 Loss: 7.9058e+07
Epoch: 28 Loss: 7.7260e+07
Epoch: 29 Loss: 7.5521e+07
Epoch: 30 Loss: 7.3839e+07
Epoch: 31 Loss: 7.2213e+07
Epoch: 32 Loss: 7.0638e+07
Epoch: 33 Loss: 6.9116e+07
Epoch: 34 Loss: 6.7642e+07
Epoch: 35 Loss: 6.6216e+07
Epoch: 36 Loss: 6.4835e+07
Epoch: 37 L

Punto 2 Regresion logistica: Convertimos los datos de la variable ob en binaria del fumador

In [92]:
patients_df['smoker'] = patients_df['smoker'].map({'yes': 1, 'no': 0})

Dividimos el conjunto de datos

In [93]:
train_df = patients_df.sample(frac=0.7, random_state=200)
rest_df = patients_df.drop(train_df.index)
val_df = rest_df.sample(frac=0.5, random_state=200)
test_df = rest_df.drop(val_df.index)

Implementamos regresion logistica:

In [97]:
def standardize(data, mean=None, std=None):
    if mean is None or std is None:
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
    standardized_data = (data - mean) / std
    return standardized_data, mean, std

In [100]:
class LogisticRegressor():
    def __init__(self, degree=1, l2=None):
        self.theta = None
        self.degree = degree
        self.l2 = l2
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit_model(self, X, Y, lr=0.01, epochs=100):
        X_poly = polynomial_features(X, self.degree)
        n, m = X_poly.shape
        self.theta = np.random.rand(m + 1, 1)
        X_c = np.hstack((np.ones((n, 1)), X_poly))
        for epoch in range(epochs):
            predictions = self.sigmoid(X_c.dot(self.theta))
            error = predictions - Y
            regularization = (self.l2 / n) * self.theta if self.l2 else 0
            self.theta -= lr * (X_c.T.dot(error) + regularization)

    def predict(self, X):
        X_poly = polynomial_features(X, self.degree)
        X_c = np.hstack((np.ones((X_poly.shape[0], 1)), X_poly))
        predictions = self.sigmoid(X_c.dot(self.theta))
        # Importante pa que el codigo no se vaya a inf o none
        return np.clip(predictions, 0.0001, 0.9999)

Evaluamos los hiperparametros basicamente igual que con regresion lineal

In [104]:
def evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations):
    best_val_loss = float('inf')
    best_hyperparameters = None
    
    X_train = train_df.drop(columns=['smoker', 'charges']).values
    Y_train = train_df[['smoker']].values
    X_val = val_df.drop(columns=['smoker', 'charges']).values
    Y_val = val_df[['smoker']].values

    X_train, mean_train, std_train = standardize(X_train)
    X_val, _, _ = standardize(X_val, mean_train, std_train)
    
    for lr in learning_rates:
        for degree in degrees:
            for reg in regularizations:
                model = LogisticRegressor(degree=degree, l2=reg)
                model.fit_model(X_train, Y_train, lr=lr)
                predictions = model.predict(X_val)
                val_loss = -np.mean(Y_val * np.log(predictions) + (1 - Y_val) * np.log(1 - predictions))
                print(f"lr={lr}, degree={degree}, reg={reg}, val_loss={val_loss}")  # Mensaje de depuración
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_hyperparameters = (lr, degree, reg)
    return best_hyperparameters, best_val_loss
    learning_rates = [0.01, 0.001]
degrees = [1, 2, 3]
regularizations = [None, 0.1, 0.01]

best_hyperparameters, best_val_loss = evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations)

print("Mejores hiperparámetros:", best_hyperparameters)
print("Pérdida de validación con los mejores hiperparámetros:", best_val_loss)

lr=0.01, degree=1, reg=None, val_loss=nan
lr=0.01, degree=1, reg=0.1, val_loss=nan
lr=0.01, degree=1, reg=0.01, val_loss=nan
lr=0.01, degree=2, reg=None, val_loss=nan
lr=0.01, degree=2, reg=0.1, val_loss=nan
lr=0.01, degree=2, reg=0.01, val_loss=nan
lr=0.01, degree=3, reg=None, val_loss=nan
lr=0.01, degree=3, reg=0.1, val_loss=nan
lr=0.01, degree=3, reg=0.01, val_loss=nan
lr=0.001, degree=1, reg=None, val_loss=nan
lr=0.001, degree=1, reg=0.1, val_loss=nan
lr=0.001, degree=1, reg=0.01, val_loss=nan
lr=0.001, degree=2, reg=None, val_loss=nan
lr=0.001, degree=2, reg=0.1, val_loss=nan
lr=0.001, degree=2, reg=0.01, val_loss=nan
lr=0.001, degree=3, reg=None, val_loss=nan
lr=0.001, degree=3, reg=0.1, val_loss=nan
lr=0.001, degree=3, reg=0.01, val_loss=nan
Mejores hiperparámetros: None
Pérdida de validación con los mejores hiperparámetros: inf


In [106]:
class LogisticRegressor():
    def __init__(self, degree=1, l2=None):
        self.theta = None
        self.degree = degree
        self.l2 = l2
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit_model(self, X, Y, lr=0.01, epochs=100):
        X_poly = polynomial_features(X, self.degree)
        n, m = X_poly.shape
        self.theta = np.zeros((m + 1, 1))  # Inicializar theta con ceros
        X_c = np.hstack((np.ones((n, 1)), X_poly))
        for epoch in range(epochs):
            predictions = self.sigmoid(X_c.dot(self.theta))
            error = predictions - Y
            regularization = (self.l2 / n) * self.theta if self.l2 else 0
            self.theta -= lr * (X_c.T.dot(error) + regularization)

    def predict(self, X):
        X_poly = polynomial_features(X, self.degree)
        X_c = np.hstack((np.ones((X_poly.shape[0], 1)), X_poly))
        predictions = self.sigmoid(X_c.dot(self.theta))
        return np.clip(predictions, 0.0001, 0.9999)

# mensajes de depuración
def evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations):
    best_val_loss = float('inf')
    best_hyperparameters = None
    
    X_train = train_df.drop(columns=['smoker', 'charges']).values
    Y_train = train_df[['smoker']].values
    X_val = val_df.drop(columns=['smoker', 'charges']).values
    Y_val = val_df[['smoker']].values

    X_train, mean_train, std_train = standardize(X_train)
    X_val, _, _ = standardize(X_val, mean_train, std_train)
    
    for lr in learning_rates:
        for degree in degrees:
            for reg in regularizations:
                model = LogisticRegressor(degree=degree, l2=reg)
                model.fit_model(X_train, Y_train, lr=lr)
                predictions = model.predict(X_val)
                val_loss = -np.mean(Y_val * np.log(predictions) + (1 - Y_val) * np.log(1 - predictions))
                print(f"lr={lr}, degree={degree}, reg={reg}, val_loss={val_loss}")
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_hyperparameters = (lr, degree, reg)
    
    return best_hyperparameters, best_val_loss
learning_rates = [0.01, 0.001]
degrees = [1, 2, 3]
regularizations = [None, 0.1, 0.01]

best_hyperparameters, best_val_loss = evaluate_hyperparameters(train_df, val_df, learning_rates, degrees, regularizations)

print("Mejores hiperparámetros:", best_hyperparameters)
print("Pérdida de validación con los mejores hiperparámetros:", best_val_loss)

lr=0.01, degree=1, reg=None, val_loss=nan
lr=0.01, degree=1, reg=0.1, val_loss=nan
lr=0.01, degree=1, reg=0.01, val_loss=nan
lr=0.01, degree=2, reg=None, val_loss=nan
lr=0.01, degree=2, reg=0.1, val_loss=nan
lr=0.01, degree=2, reg=0.01, val_loss=nan
lr=0.01, degree=3, reg=None, val_loss=nan
lr=0.01, degree=3, reg=0.1, val_loss=nan
lr=0.01, degree=3, reg=0.01, val_loss=nan
lr=0.001, degree=1, reg=None, val_loss=nan
lr=0.001, degree=1, reg=0.1, val_loss=nan
lr=0.001, degree=1, reg=0.01, val_loss=nan
lr=0.001, degree=2, reg=None, val_loss=nan
lr=0.001, degree=2, reg=0.1, val_loss=nan
lr=0.001, degree=2, reg=0.01, val_loss=nan
lr=0.001, degree=3, reg=None, val_loss=nan
lr=0.001, degree=3, reg=0.1, val_loss=nan
lr=0.001, degree=3, reg=0.01, val_loss=nan
Mejores hiperparámetros: None
Pérdida de validación con los mejores hiperparámetros: inf
